In [2]:
import pandas as pd # Data handling 
import numpy as np

import tensorflow as tf # Neural networks  
from tensorflow import keras
from tensorflow.keras import layers

import plotly.graph_objects as go # Visualization
import plotly.express as px

import sklearn as sk # Stats / ML
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import cross_val_score, train_test_split, KFold, GridSearchCV, TimeSeriesSplit
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import impute

from xgboost import XGBRegressor # Efficient Gradient Boosting

In [14]:
df_train = pd.read_csv("../data/processed/train_imputed.csv", index_col=[3])
df_test = pd.read_csv("../data/processed/test_imputed.csv", index_col=[3])
subm = pd.read_csv("../data/raw/submission_format.csv")

display(df_train.head())
display(df_test.head())

,year,weekofyear,city,total_cases,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,reanalysis_air_temp_k,reanalysis_avg_temp_k,...,reanalysis_max_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
week_start_date,,,,,,,,,,,,,,,,,,,,,
1990-04-30,1990,18,1,4,0.122600,0.103725,0.198483,0.177617,297.572857,297.742857,...,299.8,32.00,73.365714,12.42,2.628571,25.442857,6.900000,29.4,20.0,16.0
1990-05-07,1990,19,1,5,0.169900,0.142175,0.162357,0.155486,298.211429,298.442857,...,300.9,17.94,77.368571,22.82,2.371429,26.714286,6.371429,31.7,22.2,8.6
1990-05-14,1990,20,1,4,0.032250,0.172967,0.157200,0.170843,298.781429,298.878571,...,300.5,26.10,82.052857,34.54,2.300000,26.714286,6.485714,32.2,22.8,41.4
1990-05-21,1990,21,1,3,0.128633,0.245067,0.227557,0.235886,298.987143,299.228571,...,301.4,13.90,80.337143,15.36,2.428571,27.471429,6.771429,33.3,23.3,4.0
1990-05-28,1990,22,1,6,0.196200,0.262200,0.251200,0.247340,299.518571,299.664286,...,301.9,12.20,80.460000,7.52,3.014286,28.942857,9.371429,35.0,23.9,5.8


,year,weekofyear,city,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
week_start_date,,,,,,,,,,,,,,,,,,,,
2008-04-29,2008,18,1,-0.018900,-0.018900,0.102729,0.091200,298.492857,298.550000,294.527143,301.1,25.37,78.781429,78.60,3.128571,26.528571,7.057143,33.3,21.7,75.2
2008-05-06,2008,19,1,-0.018000,-0.012400,0.082043,0.072314,298.475714,298.557143,294.395714,300.8,21.83,78.230000,12.56,2.571429,26.071429,5.557143,30.0,22.2,34.3
2008-05-13,2008,20,1,-0.001500,-0.004324,0.151083,0.091529,299.455714,299.357143,295.308571,302.2,4.12,78.270000,3.66,4.428571,27.928571,7.785714,32.8,22.8,3.0
2008-05-20,2008,21,1,0.202833,-0.019867,0.124329,0.125686,299.690000,299.728571,294.402857,303.0,2.20,73.015714,0.00,4.342857,28.057143,6.271429,33.3,24.4,0.3
2008-05-27,2008,22,1,0.056800,0.039833,0.062267,0.075914,299.780000,299.671429,294.760000,302.3,4.36,74.084286,0.76,3.542857,27.614286,7.085714,33.3,23.3,84.1


## Gradient Boosting

In [19]:
xgb = XGBRegressor(objective='reg:squarederror') 
y = df_train['total_cases'].to_frame()

param = {
    'max_depth':[i for i in range(3,7)], # depth of tree
    'eta':[0.001, .01], # Step size shrinkage used in update to prevents overfitting
    'gamma':[0, 1], # Minimum loss reduction required to make a further partition on a leaf node of the tree (idk bro)
    'n_estimators':[15, 20], # Number of trees 
    'lambda':[.001, .01], # l2 regularization coefficient
    'alpha':[.001, .01] # l1 regularization coefficient
}

grid_search = GridSearchCV(
    estimator = xgb,
    param_grid = param,
    scoring = 'neg_mean_absolute_error',
    verbose = 1,
    cv = 8,
    n_jobs=-1
)

xgb = grid_search.fit(df_train.drop('total_cases', axis=1), y)

Fitting 8 folds for each of 128 candidates, totalling 1024 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 336 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done 836 tasks      | elapsed:   24.7s
[Parallel(n_jobs=-1)]: Done 1024 out of 1024 | elapsed:   29.5s finished


In [20]:
pred = xgb.predict(df_train.drop('total_cases', axis=1))
plt = go.Figure(data=[
    go.Scatter(x=df.index, y=df_train['total_cases'], mode
])

SyntaxError: invalid syntax (<ipython-input-20-cd80d873f9b0>, line 3)